In [ ]:
import numpy as np
import pandas as pd
import pickle
import yaml
import mlflow
import mlflow.sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json
from mlflow.models import infer_signature
import dagshub
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up DagsHub credentials for MLflow tracking
dagshub_token = os.getenv("DAGSHUB_PAT")
if not dagshub_token:
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

# Set up MLflow tracking URI
mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)


def load_data(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)
    df.fillna('', inplace=True)
    return df


def load_model(model_path: str):
    with open(model_path, 'rb') as file:
        return pickle.load(file)


def load_vectorizer(vectorizer_path: str) -> TfidfVectorizer:
    with open(vectorizer_path, 'rb') as file:
        return pickle.load(file)


def load_params(params_path: str) -> dict:
    with open(params_path, 'r') as file:
        return yaml.safe_load(file)


def evaluate_model(model, X_test: np.ndarray, y_test: np.ndarray):
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    cm = confusion_matrix(y_test, y_pred)
    return report, cm


def log_confusion_matrix(cm, dataset_name):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix for {dataset_name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')

    cm_file_path = f'confusion_matrix_{dataset_name}.png'
    plt.savefig(cm_file_path)
    mlflow.log_artifact(local_path=cm_file_path)
    plt.close()


def save_model_info(run_id: str, model_path: str, file_path: str) -> None:
    model_info = {
        "run_id": run_id,
        "model_path": model_path
    }
    with open(file_path, "w") as file:
        json.dump(model_info, file, indent=4)

#mlflow.lightgbm.autolog(log_input_examples=True)

# Load params
root_dir = r"D:\yt-comment-sentiment-analysis2\notebooks"
params = load_params(os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\params.yaml"))
# Load test data
test_data = load_data(
    os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2/data/interim/test_processed.csv")
)
dataset = mlflow.data.from_pandas(test_data, name="evaluation_set")

def main():
    mlflow.set_experiment("dvc-pipeline-1")

    with mlflow.start_run() as run:
        

        # Log parameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Load model and vectorizer
        model = load_model(os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\lgbm_model.pkl"))
        vectorizer = load_vectorizer(os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\tfidf_vectorizer.pkl"))

        

        X_test_tfidf = vectorizer.transform(test_data["clean_comment"].values)

        X_test_df = pd.DataFrame(
            X_test_tfidf.toarray(),
            columns=vectorizer.get_feature_names_out()
        )
        y_test = test_data["category"].values

        # Signature inference
        input_example = pd.DataFrame(
            X_test_tfidf.toarray()[:5],
            columns=vectorizer.get_feature_names_out()
        )

        signature = infer_signature(
            X_test_df[:5],
            model.predict(X_test_df[:5])
        )

        # Log model
        mlflow.lightgbm.log_model(
            model,
            name = "lgbm_model",
            signature=signature,
            input_example=input_example
        )

        # Save model info
        save_model_info(run.info.run_id, "lgbm_model", "experiment_info.json")

        # Log vectorizer
        mlflow.log_artifact(local_path=os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\tfidf_vectorizer.pkl"))

        #log data
        mlflow.log_input(dataset , context="evaluation")
        
        # Evaluate
        report, cm = evaluate_model(model, X_test_df, y_test)

        # Log metrics
        for label, metrics in report.items():
            if isinstance(metrics, dict):
                mlflow.log_metrics({
                    f"test_{label}_precision": metrics["precision"],
                    f"test_{label}_recall": metrics["recall"],
                    f"test_{label}_f1-score": metrics["f1-score"]
                })

        # Log confusion matrix
        log_confusion_matrix(cm, "Test Data")

        # Tags
        mlflow.set_tag("model_type", "LightGBM")
        mlflow.set_tag("task", "Sentiment Analysis")
        mlflow.set_tag("dataset", "YouTube Comments")


if __name__ == "__main__":
    main()


d:\yt-comment-sentiment-analysis2\myenv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run victorious-bird-318 at: https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow/#/experiments/0/runs/685e4935b5d544838f9f8ea44ae6740b
🧪 View experiment at: https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow/#/experiments/0


In [12]:
import numpy as np
import pandas as pd
import pickle
import yaml
import mlflow
import mlflow.sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json
from mlflow.models import infer_signature
import dagshub
from dotenv import load_dotenv
import logging   # 🔴 added

# -------------------- LOGGING SETUP --------------------
logger = logging.getLogger("model_evaluation")
logger.setLevel(logging.DEBUG)

if not logger.handlers:
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)

    file_handler = logging.FileHandler("model_evaluation.log")
    file_handler.setLevel(logging.DEBUG)

    formatter = logging.Formatter(
        "%(asctime)s - %(levelname)s - %(message)s"
    )
    console_handler.setFormatter(formatter)
    file_handler.setFormatter(formatter)

    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
# ------------------------------------------------------

# Load environment variables
load_dotenv()

# Set up DagsHub credentials for MLflow tracking
dagshub_token = os.getenv("DAGSHUB_PAT")
if not dagshub_token:
    logger.error("DAGSHUB_PAT environment variable is not set")
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)


def load_data(file_path: str) -> pd.DataFrame:
    logger.info(f"Loading data from {file_path}")
    df = pd.read_csv(file_path)
    df.fillna('', inplace=True)
    logger.debug(f"Data shape: {df.shape}")
    return df


def load_model(model_path: str):
    logger.info(f"Loading model from {model_path}")
    with open(model_path, 'rb') as file:
        return pickle.load(file)


def load_vectorizer(vectorizer_path: str) -> TfidfVectorizer:
    logger.info(f"Loading vectorizer from {vectorizer_path}")
    with open(vectorizer_path, 'rb') as file:
        return pickle.load(file)


def load_params(params_path: str) -> dict:
    logger.info(f"Loading params from {params_path}")
    with open(params_path, 'r') as file:
        return yaml.safe_load(file)


def evaluate_model(model, X_test: pd.DataFrame, y_test: np.ndarray):
    logger.info("Evaluating model")
    logger.debug(f"X_test shape: {X_test.shape}")
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    cm = confusion_matrix(y_test, y_pred)
    return report, cm


def log_confusion_matrix(cm, dataset_name):
    logger.info("Logging confusion matrix")
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix for {dataset_name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')

    cm_file_path = f'confusion_matrix_{dataset_name}.png'
    plt.savefig(cm_file_path)
    mlflow.log_artifact(local_path=cm_file_path)
    plt.close()


def save_model_info(run_id: str, model_path: str, file_path: str) -> None:
    logger.info("Saving model info")
    model_info = {
        "run_id": run_id,
        "model_path": model_path
    }
    with open(file_path, "w") as file:
        json.dump(model_info, file, indent=4)


# -------------------- LOAD STATIC DATA --------------------
root_dir = r"D:\yt-comment-sentiment-analysis2\notebooks"

params = load_params(
    os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\params.yaml")
)

test_data = load_data(
    os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\data\interim\test_processed.csv")
)

dataset = mlflow.data.from_pandas(test_data, name="evaluation_set")
# --------------------------------------------------------


def main():
    logger.info("Starting MLflow evaluation run")
    mlflow.set_experiment("dvc-pipeline-1")

    with mlflow.start_run() as run:
        logger.info(f"MLflow run started: {run.info.run_id}")

        for key, value in params.items():
            mlflow.log_param(key, value)

        model = load_model(
            os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\lgbm_model.pkl")
        )
        vectorizer = load_vectorizer(
            os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\tfidf_vectorizer.pkl")
        )

        X_test_tfidf = vectorizer.transform(test_data["clean_comment"].values)

        X_test_df = pd.DataFrame(
            X_test_tfidf.toarray(),
            columns=vectorizer.get_feature_names_out()
        )

        y_test = test_data["category"].values

        input_example = X_test_df[:5].astype("float32")

        signature = infer_signature(
            input_example,
            model.predict(input_example)
        )

        mlflow.lightgbm.log_model(
            model,
            name="lgbm_model",
            signature=signature,
            input_example=input_example
        )

        save_model_info(run.info.run_id, "lgbm_model", "experiment_info.json")

        mlflow.log_artifact(
            local_path=os.path.join(root_dir, r"D:\yt-comment-sentiment-analysis2\tfidf_vectorizer.pkl")
        )

        mlflow.log_input(dataset, context="evaluation")

        report, cm = evaluate_model(model, X_test_df, y_test)

        for label, metrics in report.items():
            if isinstance(metrics, dict):
                mlflow.log_metrics({
                    f"test_{label}_precision": metrics["precision"],
                    f"test_{label}_recall": metrics["recall"],
                    f"test_{label}_f1-score": metrics["f1-score"]
                })

        log_confusion_matrix(cm, "Test Data")

        mlflow.set_tag("model_type", "LightGBM")
        mlflow.set_tag("task", "Sentiment Analysis")
        mlflow.set_tag("dataset", "YouTube Comments")

        logger.info("Model evaluation completed successfully")


if __name__ == "__main__":
    main()


2026-01-28 11:43:57,926 - INFO - Loading params from D:\yt-comment-sentiment-analysis2\params.yaml
2026-01-28 11:43:57,927 - INFO - Loading data from D:\yt-comment-sentiment-analysis2\data\interim\test_processed.csv


2026-01-28 11:43:57,952 - INFO - Starting MLflow evaluation run
2026-01-28 11:43:59,425 - INFO - MLflow run started: 5bb93147e62b4ba894a71e47c57ed7dc
2026-01-28 11:44:00,111 - INFO - Loading model from D:\yt-comment-sentiment-analysis2\lgbm_model.pkl
2026-01-28 11:44:00,124 - INFO - Loading vectorizer from D:\yt-comment-sentiment-analysis2\tfidf_vectorizer.pkl
2026-01-28 11:44:30,029 - INFO - Saving model info
d:\yt-comment-sentiment-analysis2\myenv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have

🏃 View run auspicious-doe-331 at: https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow/#/experiments/0/runs/5bb93147e62b4ba894a71e47c57ed7dc
🧪 View experiment at: https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow/#/experiments/0
